<a href="https://colab.research.google.com/github/khushee-g/EEG/blob/main/Band_Pass_Exponentiated_Muse_Data_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # Linear algebra
import pandas as pd
# ModelLing
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint

# Tree Visualisation
from sklearn.tree import export_graphviz
from IPython.display import Image
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics

In [ ]:
import numpy as np
from scipy.signal import butter, filtfilt
import pandas as pd

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

fs = 256
lowcut = 0.5
highcut = 4

data = pd.read_csv('/content/mindMonitor_2024-11-15--21-52-00.csv')

num_nan_raw_AF8 = data['RAW_AF8'].isnull().sum()
num_nan_raw_AF7 = data['RAW_AF7'].isnull().sum()
num_nan_raw_TP9 = data['RAW_TP9'].isnull().sum()
num_nan_raw_TP10 = data['RAW_TP10'].isnull().sum()

if num_nan_raw_AF8 > 0:
    data['RAW_AF8'] = data['RAW_AF8'].interpolate(method='linear')

if num_nan_raw_AF7 > 0:
    data['RAW_AF7'] = data['RAW_AF7'].interpolate(method='linear')

if num_nan_raw_TP9 > 0:
    data['RAW_TP9'] = data['RAW_TP9'].interpolate(method='linear')

if num_nan_raw_TP10 > 0:
    data['RAW_TP10'] = data['RAW_TP10'].interpolate(method='linear')


filtered_delta_AF8 = bandpass_filter(data['RAW_AF8'].values, lowcut, highcut, fs, order=5)
filtered_delta_AF7 = bandpass_filter(data['RAW_AF7'].values, lowcut, highcut, fs, order=5)
filtered_delta_TP9 = bandpass_filter(data['RAW_TP9'].values, lowcut, highcut, fs, order=5)
filtered_delta_TP10 = bandpass_filter(data['RAW_TP10'].values, lowcut, highcut, fs, order=5)

print(filtered_delta_TP10)


[ 2.00767501e+73  1.80000531e+73  1.59439966e+73 ... -1.46678676e+35
 -1.26937680e+35 -1.09212825e+35]


<ipython-input-3-6408df1d375e>:21: DtypeWarning: Columns (39) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/mindMonitor_2024-11-15--21-52-00.csv')


In [ ]:
fs = 256
lowcut = 4
highcut = 8

num_nan_raw_AF7 = data['RAW_AF7'].isnull().sum()

if num_nan_raw_TP10 > 0:
    data['RAW_AF7'] = data['RAW_AF7'].interpolate(method='linear')


filtered_theta_AF7 = bandpass_filter(data['RAW_AF7'].values, lowcut, highcut, fs, order=5)


In [ ]:
#Delta AF7-AF8 coherence - corresponds to COH.A.delta.a.FP1.b.FP2

from scipy import signal

data = pd.read_csv('/content/mindMonitor_2024-11-15--21-52-00.csv')

channel_1 = filtered_delta_AF8
channel_2 = filtered_delta_AF7

#channel_1_mean = np.nanmean(channel_1)
#channel_2_mean = np.nanmean(channel_2)

#channel_1 = np.nan_to_num(channel_1, nan=np.nanmean(channel_1))
#channel_2 = np.nan_to_num(channel_2, nan=np.nanmean(channel_2))

fs = 256
f, Cxy = signal.coherence(channel_1, channel_2, fs=fs, nperseg=1024)  # fs is sampling frequency

coherence_data = pd.DataFrame({'Frequency': f, 'Coherence DeltaAF7DeltaAF8': Cxy})

coherence_AF8_AF7 = coherence_data['Coherence DeltaAF7DeltaAF8'].mean()
coherence_AF8_AF7 *=100
print(coherence_AF8_AF7)


50.97487654935996


<ipython-input-5-28087834e76c>:5: DtypeWarning: Columns (39) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/mindMonitor_2024-11-15--21-52-00.csv')


In [ ]:
#Delta AF8-TP9 coherence - corresponds to COH.A.delta.f.F4.h.T3



from scipy import signal

data = pd.read_csv('/content/mindMonitor_2024-11-15--21-52-00.csv')

channel_1 = filtered_delta_AF8
channel_1 = filtered_delta_TP9

channel_1_mean = np.nanmean(channel_1)
channel_2_mean = np.nanmean(channel_2)

channel_1 = np.nan_to_num(channel_1, nan=np.nanmean(channel_1))
channel_2 = np.nan_to_num(channel_2, nan=np.nanmean(channel_2))

fs = 256
f, Cxy = signal.coherence(channel_1, channel_2, fs=fs, nperseg=1024)  # fs is sampling frequency

coherence_data = pd.DataFrame({'Frequency': f, 'Coherence DeltaAF8DeltaTP9': Cxy})

coherence_AF8_TP9 = coherence_data['Coherence DeltaAF8DeltaTP9'].mean()
coherence_AF8_TP9 *=100
print(coherence_AF8_TP9)

51.55583751588699


<ipython-input-6-d40f442128e9>:7: DtypeWarning: Columns (39) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/mindMonitor_2024-11-15--21-52-00.csv')


In [ ]:
risk = 0
#coeff intercept + coeff channel val * (coherence*100)
#e^x


#COH.A.delta.a.FP1.b.FP2
AF7_AF8 = 0.0246*(coherence_AF8_AF7) - 1.7527
AF7_AF8 = np.exp(AF7_AF8)
AF7_AF8 -=1
risk += AF7_AF8



#COH.A.delta.f.F4.h.T3
AF8_TP9 = -0.0040*(coherence_AF8_TP9) + 0.1324
AF8_TP9 = np.exp(AF8_TP9)
AF8_TP9 -=1
risk += AF8_TP9



#AB.C.alpha.q.T6
TP10 = 0.0097*filtered_delta_TP10.mean() - 0.1531
TP10 = np.exp(TP10)
TP10 -=1
risk += TP10



#AB.B.theta.c.F7
AF7 = -0.0152 *filtered_theta_AF7.mean() + 0.2759
AF7 = np.exp(AF7)
AF7 -=1
risk += AF7


print(risk)

if risk > 0:
  prediction_val = (risk-1)*100
  print("You are " + str(prediction_val) + "% more likely to have behavioral disorder")

else:
  prediction_val = (risk+1)*-100
  print("You are " + str(prediction_val) + " % less likely to have behavioral disorder")




-1.1460808076429658
You are 14.608080764296583 % less likely to have behavioral disorder


In [ ]:
!pip install marginaleffects

import numpy as np
import polars as pl
from marginaleffects import *
import statsmodels.formula.api as smf
mtcars = pl.read_csv("/content/mindMonitor_2024-11-15--21-52-00.csv")
mod = smf.ols("mpg ~ hp * wt * am", data = mtcars).fit()

print(mod.summary().as_text())